# <center> ML-7. Оптимизация гиперпараметров модели

In [3]:
#Импорты

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import sklearn.linear_model as linear_model

NameError: name 'X_train_scaled_poly' is not defined